In [2]:
!pip install mistralai


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 KB 5.3 MB/s eta 0:00:000:00:01m eta 0:00:01


In [2]:
# Obtenir une version json de la sortie du LLM
import json
def to_json(chat_response):
    j_end = chat_response.find('}') + 1
    j_part = chat_response[:j_end]

    j = json.loads(j_part)
    
    if j_end == len(chat_response):
        return j, "rien"
        
    t_part = chat_response[j_end:].lstrip()
    return j, t_part

In [3]:
from datetime import datetime

memory_message = []

def action_response(action, filenames, user_prompt, assistant_response):
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    
    match action:
        case "create" :
            nbr = len(filenames)
            for filename in filenames:
                print(f"   <--création du fichier '{filename}'-->")
            summary = f'{now} : Création de {", ".join(filenames)}, {nbr} fichier(s)'

        case "delete" :
            nbr = len(filenames)
            for filename in filenames:
                print(f"   <--suppression du fichier '{filename}'-->")
            summary = f'{now} : Suppression de {", ".join(filenames)}, {nbr} fichier(s)'
        case _:
            summary = f'{now} : Échange sans action :  {user_prompt} → {assistant_response}'

    memory_message.append(summary)
    if len(memory_message) > 10:
        memory_message.pop(0)

In [4]:
from mistralai import Mistral

API_KEY = ""
MODEL = "mistral-tiny"

client = Mistral(api_key=API_KEY)

def s_prompt():
    memory = ",\n ".join(memory_message)
    system_prompt = (
        "Tu es un assistant polyvalent capable de :"
    
        "1. Créer ou supprimer des fichiers selon les commandes reçues."
        "2. Répondre à des questions ou discuter simplement si aucune action sur les fichiers n’est demandée, exclusivement en français."
    
        "Tu dois :"
        "- Répondre uniquement avec un JSON structuré (jamais en dehors)."
        "- Être précis et concis, sans explication supplémentaire."
        "- Utilise un ton naturel dans le champ 'response' (exemples : 'Je m’en occupe', 'C’est bon pour moi', 'Très bien', etc.), sans répéter les actions comme 'fichier créé'"
        "- Déduire l'action correcte ou répondre normalement si aucun fichier n’est concerné."
    
        "Format de la réponse JSON :"
        "{"
          "'action': 'create' | 'delete' | 'none',"
          "'filename': ['nom_du_fichier.txt', ...],"
          "'file_number': int,"
          "'response': 'Message court et naturel'"
        "}" 
        ""
        ""
        "Voici une mémoire des derniers échanges :"
        + memory
    )
    return system_prompt



def prompt_to_llm(prompt):
    system_prompt = s_prompt()
    messages = [{"role": "system", "content": system_prompt}]
    
    messages.append({"role": "user", "content": prompt})
    
    chat_response = client.chat.complete(
        model=MODEL,
        messages = messages
    )

    response_content = chat_response.choices[0].message.content 
    return (chat_response, response_content)

In [5]:
from datetime import datetime

maintenant = datetime.now()
print(maintenant)


2025-08-24 16:57:30.733159


In [6]:
# Activer notre chatbot sur le terminal pour lui demander des choses

def chatbot():
    print(f"Bienvenue sur votre ChatBot. Vous pouvez discuter avec lui mais aussi gérer vos fichiers (création et suppréssion).")
    while True:
        user_input = input('Vous : ')

        if user_input.lower() in ["quit","exit","q"]:
            print(f"Merci. À une prochaine fois.")
            break

        chat_response, response = prompt_to_llm(user_input)  # envoie de l'input au LLM et récupération de sa réponse brut + content
        print("Chatbot : ",end="")  # chatbot s'apprête à répondre
        j_part, t_part = to_json(response)

        if t_part != 'rien':
            print(t_part)
        print(j_part['response'])
        
        action_response(j_part['action'], j_part['filename'], user_input, j_part['response']) 


        
        


In [7]:
chatbot()

Bienvenue sur votre ChatBot. Vous pouvez discuter avec lui mais aussi gérer vos fichiers (création et suppréssion).


Vous :  q


Merci. À une prochaine fois.
